In [19]:
import numpy as np
import pandas as pd
import fasttext
import os
import os.path
from sklearn.metrics import precision_score, recall_score, f1_score 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [20]:
dirs = os.getcwd()
len(os.listdir(dirs))

346

In [21]:
import re
zhPattern = re.compile(u'[\u4e00-\u9fa5]+')
numPattern = re.compile('[0-9]')
def deal(x):
    if x == -888:  #这是缺失值填充的（记得先填充缺失值）
        return x
    contents = str(x)
    zhmatch = zhPattern.search(contents)
    if zhmatch:  
        return -666  #汉字 直接返回
    numbermatch = numPattern.search(contents)
    if numbermatch:
        return 8 #表示 数字 
    else:
        return 9 #表示字母
        

In [22]:
def prepross(data):
    #1 使用填充的方式把
    dataPro = data.copy()
    label = dataPro.columns
    #统计每行的缺失率
    dataPro.insert(0,"missingRate",0)
    dataPro["missingRate"] = dataPro.isnull().sum(axis = 1)
    
    dataPro.fillna(-888,inplace = True)
    # 数字和空缺值填充，保留分布格局
    for i in range(len(label)):
        dataPro[label[i]] = dataPro[label[i]].apply(lambda x: deal(x) )
        
    dataPro.insert(1,"index",dataPro.index)
    dataPro.insert(2,"position",0)
    dataPro.insert(3,"first_last_No",0)
    dataPro.insert(4,"class2_number",0)
    dataPro.insert(5,"alphabet",0)
    
    number1 = 0
    for i in range(dataPro.shape[0]):
        if i < int(dataPro.shape[0]/3):
            dataPro["position"][i] = 1
        elif i < int(2*dataPro.shape[0]/3) and i >= int(dataPro.shape[0]/3):
            dataPro["position"][i] = 2
        else:
            dataPro["position"][i] = 3

        # 下面表示的是 头部加上序列号（2标签可能全是数字），尾部全部用 -1
        #  (dataPro.iloc[i,:] == -666).sum() 比较保险，data[i] 出错。
        if (dataPro.iloc[i,:] == 8).sum() < (dataPro.iloc[i,:] == -666).sum() and number1 == 0:
                dataPro["first_last_No"][i] = i
        else:
             #针对 class2 全是数字的情况
            if i < int(dataPro.shape[0]/3):
                if (dataPro.iloc[i+1,:] == 8).sum() < (dataPro.iloc[i+1,:] == -666).sum()and number1 == 0: 
                    dataPro["first_last_No"][i] = i  #仍然编码 ，原来是i
                    dataPro["class2_number"][i] = 1 
                else:    
                    number1 += 1
                    
        if (dataPro.iloc[i,:] == 8).sum() < (dataPro.iloc[i,:] == -666).sum() and number1 > 0:
               dataPro["first_last_No"][i] = -1 
                
        if (dataPro.iloc[i,:] == 9).sum() > (dataPro.iloc[i,:] == -666).sum() and i < int(dataPro.shape[0]/3):
            dataPro["alphabet"][i] = 1
    return dataPro

In [ ]:
onlineTest = True

if onlineTest == True:
    train =  pd.read_csv("../trainFullNumberNew8.csv",encoding = "utf-8",header = None)
    print(".............")
    l = 56
    trainx = train.iloc[:,1:l]  # 取了20维度，效果明显提升。明天试下多少合适(用循环语句)
    trainy = train.iloc[:,0]
    trainx.fillna(-888,inplace = True)
    trainx.columns = np.arange(0,l-1)
    print("trainx",trainx.head())
    xgb1 = XGBClassifier(
                learning_rate =0.1,
        #         n_estimators=mm.shpa[0],
                max_depth=5,
                min_child_weight=5,
                gamma=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                nthread=3,
                scale_pos_weight=1,
                StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
                seed=1)
    xgb1.fit(trainx,trainy)
    print(trainx.shape,trainy.shape)
rightNumber = 0

number = 0
fulldata = 0
kk = 0

for path,dirname,filenames in os.walk(dirs): 
    for filename in filenames:
        if filename.endswith('.csv'):  #因为有ipython 在所以必须有这句
            number += 1
#             print("filename",filename)
            data = pd.read_csv(filename,encoding = "gb18030",header = None)
            cc = prepross(data)
            tmp = 0
            cc.columns = np.arange(0,cc.shape[1])
            tmp = pd.concat([trainx,cc],axis = 0, ignore_index = True)
            cc1 = tmp.iloc[trainx.shape[0]:,0:l-1]
            cc1.fillna(-888,inplace = True)
            prexgb = xgb1.predict(cc1)
            print("preshape",data.shape)
            data.insert(0,"test_result",prexgb)  
            print("aftershape",data.shape)
            name = filename.split(".")[0]
            finalpath = "../test_result"
            newpath = os.path.join(finalpath,name)+".csv"
            data.to_csv(newpath,index = False,header = None,encoding ="gb18030")
            if number % 10 == 0:
                print(number)

.............
trainx    0   1   2   3   4   5    6      7      8      9   ...       45     46  \
0  65   0   1   0   0   0 -666 -888.0 -888.0 -888.0  ...   -888.0 -888.0   
1  65   1   1   1   0   0 -888 -888.0 -888.0 -888.0  ...   -888.0 -888.0   
2  56   2   1   2   0   0 -888 -888.0 -888.0 -888.0  ...   -888.0 -666.0   
3   8   3   1   3   0   0 -666 -666.0 -666.0 -666.0  ...   -666.0 -666.0   
4   8   4   1   0   0   1 -666 -666.0    9.0    9.0  ...      9.0    9.0   

      47     48     49     50     51     52     53     54  
0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0  
1 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0 -888.0  
2 -888.0 -888.0 -888.0 -888.0 -888.0 -666.0 -888.0 -666.0  
3 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0 -666.0  
4    9.0    9.0    9.0    9.0    9.0    9.0    8.0    9.0  

[5 rows x 55 columns]
(56710, 55) (56710,)


d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set o

preshape (181, 8)
aftershape (181, 9)
name addcsvmultiple1_ok
name ../test_result\addcsvmultiple1_ok.csv


d:\gongju\python3.6 64\anzhuang\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


preshape (502, 55)
aftershape (502, 56)
name csvsingle10_ok
name ../test_result\csvsingle10_ok.csv


...

preshape (388, 53)
aftershape (388, 54)
name csvsingle23_ok
name ../test_result\csvsingle23_ok.csv


d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


preshape (473, 67)
aftershape (473, 68)
name csvsingle28_ok
name ../test_result\csvsingle28_ok.csv


d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


preshape (603, 53)
aftershape (603, 54)
name csvsingle3_ok
name ../test_result\csvsingle3_ok.csv


In [ ]:
a = pd.DataFrame([[1,5,6],[5,6,8]],columns = ['a','1','5'])
b = pd.DataFrame([[np.nan,6,np.nan],[9,np.nan,8],[5,6,8],[np.nan,np.nan,np.nan]],columns = ['1','5','e'])
print(a,'\n',b)
# pd.concat([a,b],axis = 0,ignore_index = False)
# np.arange(0,15)
# if onlineTest = False:
#     
x = b.isnull().sum(axis = 1)
# data =  pd.read_csv("../trainFullNumberNew8.csv",encoding = "utf-8",header = None)
